In [ ]:
#importing libraries
import pandas as pd
import pyodbc
import re

In [ ]:
# establish a connection to the SQL server using Windows authentication
server_name = 'server'
database_name = 'database'

conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};' +
                      f'SERVER={server_name};' +
                      f'DATABASE={database_name};' +
                      'Trusted_Connection=yes;')

In [ ]:
#Creating the tables to be used in the ssis ETl package

#denormalized table
CREATE TABLE new_es_requests(
    [es_key] [int] IDENTITY(1,1) PRIMARY KEY NOT NULL,
    [esrequest_id] [int] NULL,
    [user_id] [int] NULL,
    [faculty_accounts] [smallint] NULL,
    [student_accounts] [smallint] NULL,
    [student_accounts_created] [varchar](8000) NULL,
    [sas_user_account] [varchar](8000) NULL,
    [why_question] [varchar](8000) NULL,
    [where_question] [varchar](8000) NULL,
    [platform_id] [int] NULL,
    [platform_Name] [nvarchar](255) NULL,
    [platform_comment] [varchar](8000) NULL,
    [platform_services_list] [varchar](8000) NULL,
    [course_name] [nvarchar](255) NULL,
    [institution_id] [int] NULL,
    [institution_name] [nvarchar](255) NULL,
    [institution_url] [nvarchar](255) NULL,
    [first_name] [nvarchar](255) NULL,
    [last_name] [nvarchar](255) NULL,
    [faculty_username] [nvarchar](255) NULL,
    [faculty_Password] [nvarchar](255) NULL,
    [latitude] [decimal](8, 6) NULL,
    [longitude] [decimal](9, 6) NULL,
    [updated_at] [datetime] NULL,
    [fulfilled_at] [datetime] NULL
);

#accounts table to hold the account ranges extrated from the notes column
CREATE TABLE new_accounts(
    [account_key] [int] IDENTITY(1,1) PRIMARY KEY NOT NULL,
    [esrequest_id] [int] NULL,
    [start_range] [nvarchar](255) NULL,
    [end_range] [nvarchar](255) NULL
);

#final account table with the individual student accounts
CREATE TABLE user_accounts(
    [user_account_id] [int] IDENTITY(1,1) PRIMARY KEY NOT NULL,
    [username] [varchar](15) NULL,
    [esrequest_id] [int] NULL);

In [ ]:
#Dropping the tables before data loading
TRUNCATE TABLE new_es_requests;

TRUNCATE TABLE new_accounts;

TRUNCATE TABLE user_accounts;


In [ ]:
#Extracting data to be loaded into the denormalized table
SELECT esrequest_id, E.user_id, faculty_accounts, student_accounts, student_accounts_created, sas_user_account,why_question, where_question, platform_id, P.name AS platform_Name, platform_comment, platform_services_list,
course_name, institution_id, I.name AS institution_name, url AS institution_url, first_name, last_name,
FA.username AS faculty_username, FA.password AS faculty_Password,latitude, longitude, E.updated_at,
E.fulfilled_at
FROM esrequests E FULL JOIN users U
ON E.id = U.id
FULL JOIN institutions I
ON U.id = I.id
FULL JOIN esrequest_platform EP
ON EP.esrequest_id = E.id
FULL JOIN platforms P
ON EP.platform_id = P.id
FULL JOIN faculty_accounts FA
ON U.id = FA.id
WHERE esrequest_id IS NOT NULL
ORDER BY esrequest_id

In [ ]:
#cleaning, transforming, and extracting the account ranges to be loaded into the accounts table 
SELECT b.id, SUBSTRING(Start_Range, 2, 9) AS Start_Range, SUBSTRING(End_Range, 1, 9) AS End_Range
FROM
(SELECT a.id,
REVERSE(PARSENAME(REPLACE(REVERSE(extracted_text), '-', '.'), 1)) AS [Start_Range],
REVERSE(PARSENAME(REPLACE(REVERSE(extracted_text), '-', '.'), 2)) AS [End_Range]
FROM
(SELECT id, SUBSTRING(note, 71, 21) AS extracted_text
FROM esrequests
WHERE note IS NOT NULL AND created_at >= '2022-11-08 22:45:06.000')a)b

In [ ]:
# python code to extract individual student accounts from the ranges
# string variable to store all columns from new_accounts table
student_query = "SELECT * FROM new_accounts;"

# read the SQL query into a pandas dataframe
student_df = pd.read_sql(student_query, conn)

#delete only rows having NULL in both the ranges
student_df.dropna(subset=['start_range', 'end_range'], how='all', inplace=True)

#Create an empty dataframe for the individual usernames
usernames_df = pd.DataFrame(columns=['username'])

for index, row in student_df.iterrows():
    # extract the start and end range from the row
    begin = row['start_range']
    end = row['end_range']

    # extract the numeric portion of the start and end range using regex
    start_match = re.search(r'\d+', begin)
    start_num = int(start_match.group()) if start_match else None
    end_match = re.search(r'\d+', str(end)) if pd.notnull(end) else start_match
    end_num = int(end_match.group()) if end_match else start_num

    # generate a list of all usernames in the range
    try:
        usernames = [f"{begin[:4]}{num:04}" for num in range(start_num, end_num+1)]
    except TypeError:
        pass

    # add the usernames to the dataframe
    usernames_df = usernames_df.append(pd.DataFrame({'username': usernames}), ignore_index=True)

print(usernames_df)